### Let us learn Selenium with Python
- Selenium is a powerful tool for controlling web browsers through programs and performing browser automation.
- It is functional for all browsers, works on all major OS and its scripts are written in various languages i.e Python, Java, C#, etc

### 1. Data extraction
#### 1.1 Import libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import pandas as pd

# install the libraries for webpages interactions
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

We will try to replicate this tutorial by David Zerpa on his project [here](https://www.linkedin.com/feed/update/urn:li:activity:7196180482084986880?utm_source=share&utm_medium=member_desktop)

#### 1.2 Creating services in Google Chrome to run session called 'Driver'

In [2]:
def setup_selenium():
    chromedriver_autoinstaller.install()
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service(executable_path=chromedriver_autoinstaller.install()) 
    driver = webdriver.Chrome(service=service, options=options)
    return driver


#### 1.3 Creating a function to extract data from the website

In [3]:
url = "https://www.flashscore.com"

In [4]:
def accept_cookies(driver):
    try:
        driver.get(url)

        cookie_button = WebDriverWait(driver, 10).until (
            EC.element_to_be_clickable((BY.ID, "onetrust-accept-btn-handler"))
        )
        cookie_button.click()
        print("Cookies accepted successfully")
    except Exception as e:
            print("Cookies button not found")
            print(e)

![imageParse](laliga_league.png)

In [5]:
css_1 = "#my-leagues-list"
# css_2 is the laliga link
css_2 = "#my-leagues-list > div:nth-child(1) > div:nth-child(6) > a > span.leftMenu__text"
url_2 = "https://www.flashscore.com/football/spain/laliga/"

def navigate_to_page(driver, url):

    accept_cookies(driver)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_1))
        )
        # use javascript to click on the element
        link_1 = driver.find_element(By.CSS_SELECTOR, css_2)
        driver.execute_script("arguments[0].click();", link_1)

        print(f"Successfully navigated to {url_2}")
    except Exception as e:
        print(f"Failed to navigate, error {e}")

![imageParse](laliga_league2.png)

In [6]:
def click_results_tab(driver):
    try:
        results_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "li2"))
        )
        results_tab.click()
        print("Results tab clicked successfully")
    except Exception as e:
        print(f"Failed to click results tab, error {e}")

In [7]:
def click_show_more_matches(driver):
    try:
        while True:
            try:
                WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.event__more.event__more--static"))
                )
                show_more_button = driver.find_element(By.CSS_SELECTOR, "a.event__more.event__more--static")
                show_more_button.click()

                print("Show more matches clicked successfully")
            except ElementClickInterceptedException:
                show_more_button = driver.find_element(By.CSS_SELECTOR, "a.event__more.event__more--static")
                driver.execute_script("arguments[0].click();", show_more_button)
            except StaleElementReferenceException:
                print("Encountered a stale element, retrying...")
                continue
    except TimeoutException:
        pass
    except Exception as e:
        print(f"Failed to click show more matches, error {e}")

In [ ]:
def extract_match_data (driver, season_text):
    matches = []
    current_round = None

    elements = driver.find_elements(By.CSS_SELECTOR, "div.event__match.event__match--static")